In [ ]:
!pip install transformers

glove

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, Conv1D, GlobalMaxPooling1D, Concatenate
from tensorflow.keras.initializers import Constant

# Parameters for the model and training
max_features = 20000  # Number of words to consider as features
maxlen = 200  # Max length of individual reviews
embedding_dim = 100  # Dimension of word embeddings
lstm_units = 128  # Number of LSTM units
filters = 64  # Number of filters for CNN
kernel_size = 5  # Kernel size for CNN
batch_size = 32
epochs = 5

# Load train and test datasets from CSV files
train_data = pd.read_csv('/kaggle/input/plmdataset/train.csv/train.csv')
test_data = pd.read_csv('/kaggle/input/plmdataset/test.csv/test.csv')

# Preprocess train data
X_train = train_data['review'].values
y_train = train_data['sentiment'].values

# Preprocess test data
X_test = test_data['review'].values
y_test = test_data['sentiment'].values

# Convert labels to numerical format
y_train[y_train == 'negative'] = 0
y_train[y_train == 'positive'] = 1
y_train = y_train.astype(int)

y_test[y_test == 'negative'] = 0
y_test[y_test == 'positive'] = 1
y_test = y_test.astype(int)

# Tokenize and pad sequences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train)

X_train_tokenized = tokenizer.texts_to_sequences(X_train)
X_test_tokenized = tokenizer.texts_to_sequences(X_test)
X_train = pad_sequences(X_train_tokenized, maxlen=maxlen)
X_test = pad_sequences(X_test_tokenized, maxlen=maxlen)

# Load GloVe embeddings
embeddings_index = {}
with open('/kaggle/input/glove6/glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Create embedding matrix
num_words = min(max_features, len(tokenizer.word_index) + 1)
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i >= max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Define the model using functional API
inputs = Input(shape=(maxlen,))
embedding_layer = Embedding(num_words, embedding_dim,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=maxlen,
                            trainable=False)(inputs)

# LSTM branch
lstm_branch = LSTM(lstm_units, dropout=0.2, recurrent_dropout=0.2)(embedding_layer)

# CNN branch
cnn_branch = Conv1D(filters, kernel_size, activation='relu')(embedding_layer)
cnn_branch = GlobalMaxPooling1D()(cnn_branch)

# Concatenate both branches
merged = Concatenate()([lstm_branch, cnn_branch])
output = Dense(1, activation='sigmoid')(merged)

# Create model
model = Model(inputs=inputs, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test, y_test)
print('Test accuracy:', accuracy)



2024-02-25 15:37:09.997544: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-25 15:37:09.997674: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-25 15:37:10.117544: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Epoch 1/5


I0000 00:00:1708875476.157085     111 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


750/750 [==============================] - 190s 246ms/step - loss: 0.4559 - accuracy: 0.7809 - val_loss: 0.3434 - val_accuracy: 0.8502
Epoch 2/5
750/750 [==============================] - 183s 244ms/step - loss: 0.3137 - accuracy: 0.8662 - val_loss: 0.3127 - val_accuracy: 0.8678
Epoch 3/5
750/750 [==============================] - 184s 246ms/step - loss: 0.2607 - accuracy: 0.8920 - val_loss: 0.2990 - val_accuracy: 0.8743
Epoch 4/5
750/750 [==============================] - 183s 244ms/step - loss: 0.2167 - accuracy: 0.9153 - val_loss: 0.3503 - val_accuracy: 0.8443
Epoch 5/5
625/625 [==============================] - 29s 46ms/step - loss: 0.3171 - accuracy: 0.8644
Test accuracy: 0.8644000291824341


In [4]:
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, Conv1D, GlobalMaxPooling1D, Concatenate
from tensorflow.keras.initializers import Constant

# Record start time
start_time = time.time()
# Parameters for the model and training
max_features = 20000  # Number of words to consider as features
maxlen = 200  # Max length of individual reviews
embedding_dim = 200  # Dimension of word embeddings
lstm_units = 128  # Number of LSTM units
filters = 64  # Number of filters for CNN
kernel_size = 5  # Kernel size for CNN
batch_size = 32
epochs = 5

# Load train and test datasets from CSV files
train_data = pd.read_csv('/kaggle/input/plmdataset/train.csv/train.csv')
test_data = pd.read_csv('/kaggle/input/plmdataset/test.csv/test.csv')

# Preprocess train data
X_train = train_data['review'].values
y_train = train_data['sentiment'].values

# Preprocess test data
X_test = test_data['review'].values
y_test = test_data['sentiment'].values

# Convert labels to numerical format
y_train[y_train == 'negative'] = 0
y_train[y_train == 'positive'] = 1
y_train = y_train.astype(int)

y_test[y_test == 'negative'] = 0
y_test[y_test == 'positive'] = 1
y_test = y_test.astype(int)

# Tokenize and pad sequences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train)

X_train_tokenized = tokenizer.texts_to_sequences(X_train)
X_test_tokenized = tokenizer.texts_to_sequences(X_test)
X_train = pad_sequences(X_train_tokenized, maxlen=maxlen)
X_test = pad_sequences(X_test_tokenized, maxlen=maxlen)

# Load GloVe embeddings
embeddings_index = {}
with open('/kaggle/input/glove6/glove.6B.200d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Create embedding matrix
num_words = min(max_features, len(tokenizer.word_index) + 1)
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i >= max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Define the model using functional API
inputs = Input(shape=(maxlen,))
embedding_layer = Embedding(num_words, embedding_dim,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=maxlen,
                            trainable=False)(inputs)

# LSTM branch
lstm_branch = LSTM(lstm_units, dropout=0.2, recurrent_dropout=0.2)(embedding_layer)

# CNN branch
cnn_branch = Conv1D(filters, kernel_size, activation='relu')(embedding_layer)
cnn_branch = GlobalMaxPooling1D()(cnn_branch)

# Concatenate both branches
merged = Concatenate()([lstm_branch, cnn_branch])
output = Dense(1, activation='sigmoid')(merged)

# Create model
model = Model(inputs=inputs, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test, y_test)
print('Test accuracy:', accuracy)
end_time = time.time()

print("Time required to fine-tune: ", end_time - start_time)

Epoch 1/5
750/750 [==============================] - 174s 228ms/step - loss: 0.4101 - accuracy: 0.8048 - val_loss: 0.3090 - val_accuracy: 0.8718
Epoch 2/5
750/750 [==============================] - 173s 230ms/step - loss: 0.2788 - accuracy: 0.8844 - val_loss: 0.2806 - val_accuracy: 0.8803
Epoch 3/5
750/750 [==============================] - 169s 226ms/step - loss: 0.2140 - accuracy: 0.9150 - val_loss: 0.2992 - val_accuracy: 0.8708
Epoch 4/5
750/750 [==============================] - 169s 226ms/step - loss: 0.1577 - accuracy: 0.9435 - val_loss: 0.2790 - val_accuracy: 0.8828
Epoch 5/5
625/625 [==============================] - 27s 43ms/step - loss: 0.2910 - accuracy: 0.8835
Test accuracy: 0.8835499882698059
Time required to fine-tune:  921.8578188419342


In [5]:
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, Conv1D, GlobalMaxPooling1D, Concatenate
from tensorflow.keras.initializers import Constant

# Record start time
start_time = time.time()
# Parameters for the model and training
max_features = 20000  # Number of words to consider as features
maxlen = 200  # Max length of individual reviews
embedding_dim = 300  # Dimension of word embeddings
lstm_units = 128  # Number of LSTM units
filters = 64  # Number of filters for CNN
kernel_size = 5  # Kernel size for CNN
batch_size = 32
epochs = 5

# Load train and test datasets from CSV files
train_data = pd.read_csv('/kaggle/input/plmdataset/train.csv/train.csv')
test_data = pd.read_csv('/kaggle/input/plmdataset/test.csv/test.csv')

# Preprocess train data
X_train = train_data['review'].values
y_train = train_data['sentiment'].values

# Preprocess test data
X_test = test_data['review'].values
y_test = test_data['sentiment'].values

# Convert labels to numerical format
y_train[y_train == 'negative'] = 0
y_train[y_train == 'positive'] = 1
y_train = y_train.astype(int)

y_test[y_test == 'negative'] = 0
y_test[y_test == 'positive'] = 1
y_test = y_test.astype(int)

# Tokenize and pad sequences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train)

X_train_tokenized = tokenizer.texts_to_sequences(X_train)
X_test_tokenized = tokenizer.texts_to_sequences(X_test)
X_train = pad_sequences(X_train_tokenized, maxlen=maxlen)
X_test = pad_sequences(X_test_tokenized, maxlen=maxlen)

# Load GloVe embeddings
embeddings_index = {}
with open('/kaggle/input/glove6/glove.6B.300d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Create embedding matrix
num_words = min(max_features, len(tokenizer.word_index) + 1)
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i >= max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Define the model using functional API
inputs = Input(shape=(maxlen,))
embedding_layer = Embedding(num_words, embedding_dim,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=maxlen,
                            trainable=False)(inputs)

# LSTM branch
lstm_branch = LSTM(lstm_units, dropout=0.2, recurrent_dropout=0.2)(embedding_layer)

# CNN branch
cnn_branch = Conv1D(filters, kernel_size, activation='relu')(embedding_layer)
cnn_branch = GlobalMaxPooling1D()(cnn_branch)

# Concatenate both branches
merged = Concatenate()([lstm_branch, cnn_branch])
output = Dense(1, activation='sigmoid')(merged)

# Create model
model = Model(inputs=inputs, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test, y_test)
print('Test accuracy:', accuracy)
end_time = time.time()

print("Time required to fine-tune: ", end_time - start_time)

Epoch 1/5
750/750 [==============================] - 179s 235ms/step - loss: 0.3902 - accuracy: 0.8217 - val_loss: 0.3019 - val_accuracy: 0.8753
Epoch 2/5
750/750 [==============================] - 173s 231ms/step - loss: 0.2528 - accuracy: 0.8959 - val_loss: 0.2744 - val_accuracy: 0.8883
Epoch 3/5
750/750 [==============================] - 174s 232ms/step - loss: 0.1846 - accuracy: 0.9302 - val_loss: 0.2635 - val_accuracy: 0.8925
Epoch 4/5
750/750 [==============================] - 173s 230ms/step - loss: 0.1206 - accuracy: 0.9601 - val_loss: 0.2942 - val_accuracy: 0.8870
Epoch 5/5
625/625 [==============================] - 27s 44ms/step - loss: 0.3178 - accuracy: 0.8789
Test accuracy: 0.8789499998092651
Time required to fine-tune:  1006.1718916893005


50d

In [7]:
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, Conv1D, GlobalMaxPooling1D, Concatenate
from tensorflow.keras.initializers import Constant

# Record start time
start_time = time.time()
# Parameters for the model and training
max_features = 20000  # Number of words to consider as features
maxlen = 200  # Max length of individual reviews
embedding_dim = 50 # Dimension of word embeddings
lstm_units = 128  # Number of LSTM units
filters = 64  # Number of filters for CNN
kernel_size = 5  # Kernel size for CNN
batch_size = 32
epochs = 5

# Load train and test datasets from CSV files
train_data = pd.read_csv('/kaggle/input/plmdataset/train.csv/train.csv')
test_data = pd.read_csv('/kaggle/input/plmdataset/test.csv/test.csv')

# Preprocess train data
X_train = train_data['review'].values
y_train = train_data['sentiment'].values

# Preprocess test data
X_test = test_data['review'].values
y_test = test_data['sentiment'].values

# Convert labels to numerical format
y_train[y_train == 'negative'] = 0
y_train[y_train == 'positive'] = 1
y_train = y_train.astype(int)

y_test[y_test == 'negative'] = 0
y_test[y_test == 'positive'] = 1
y_test = y_test.astype(int)

# Tokenize and pad sequences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train)

X_train_tokenized = tokenizer.texts_to_sequences(X_train)
X_test_tokenized = tokenizer.texts_to_sequences(X_test)
X_train = pad_sequences(X_train_tokenized, maxlen=maxlen)
X_test = pad_sequences(X_test_tokenized, maxlen=maxlen)

# Load GloVe embeddings
embeddings_index = {}
with open('/kaggle/input/glove6/glove.6B.50d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Create embedding matrix
num_words = min(max_features, len(tokenizer.word_index) + 1)
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i >= max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Define the model using functional API
inputs = Input(shape=(maxlen,))
embedding_layer = Embedding(num_words, embedding_dim,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=maxlen,
                            trainable=False)(inputs)

# LSTM branch
lstm_branch = LSTM(lstm_units, dropout=0.2, recurrent_dropout=0.2)(embedding_layer)

# CNN branch
cnn_branch = Conv1D(filters, kernel_size, activation='relu')(embedding_layer)
cnn_branch = GlobalMaxPooling1D()(cnn_branch)

# Concatenate both branches
merged = Concatenate()([lstm_branch, cnn_branch])
output = Dense(1, activation='sigmoid')(merged)

# Create model
model = Model(inputs=inputs, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test, y_test)
print('Test accuracy:', accuracy)
end_time = time.time()

print("Time required to fine-tune: ", end_time - start_time)

Epoch 1/5
750/750 [==============================] - 183s 240ms/step - loss: 0.5043 - accuracy: 0.7503 - val_loss: 0.4002 - val_accuracy: 0.8260
Epoch 2/5
750/750 [==============================] - 180s 240ms/step - loss: 0.3671 - accuracy: 0.8388 - val_loss: 0.3576 - val_accuracy: 0.8438
Epoch 3/5
750/750 [==============================] - 175s 233ms/step - loss: 0.3198 - accuracy: 0.8652 - val_loss: 0.3614 - val_accuracy: 0.8397
Epoch 4/5
750/750 [==============================] - 175s 233ms/step - loss: 0.2858 - accuracy: 0.8797 - val_loss: 0.3447 - val_accuracy: 0.8517
Epoch 5/5
625/625 [==============================] - 27s 43ms/step - loss: 0.3350 - accuracy: 0.8525
Test accuracy: 0.8524500131607056
Time required to fine-tune:  955.3397836685181


10 epochs

In [ ]:
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, Conv1D, GlobalMaxPooling1D, Concatenate
from tensorflow.keras.initializers import Constant

# Record start time
start_time = time.time()
# Parameters for the model and training
max_features = 20000  # Number of words to consider as features
maxlen = 200  # Max length of individual reviews
embedding_dim = 300  # Dimension of word embeddings
lstm_units = 128  # Number of LSTM units
filters = 64  # Number of filters for CNN
kernel_size = 5  # Kernel size for CNN
batch_size = 32
epochs = 10

# Load train and test datasets from CSV files
train_data = pd.read_csv('/kaggle/input/plmdataset/train.csv/train.csv')
test_data = pd.read_csv('/kaggle/input/plmdataset/test.csv/test.csv')

# Preprocess train data
X_train = train_data['review'].values
y_train = train_data['sentiment'].values

# Preprocess test data
X_test = test_data['review'].values
y_test = test_data['sentiment'].values

# Convert labels to numerical format
y_train[y_train == 'negative'] = 0
y_train[y_train == 'positive'] = 1
y_train = y_train.astype(int)

y_test[y_test == 'negative'] = 0
y_test[y_test == 'positive'] = 1
y_test = y_test.astype(int)

# Tokenize and pad sequences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train)

X_train_tokenized = tokenizer.texts_to_sequences(X_train)
X_test_tokenized = tokenizer.texts_to_sequences(X_test)
X_train = pad_sequences(X_train_tokenized, maxlen=maxlen)
X_test = pad_sequences(X_test_tokenized, maxlen=maxlen)

# Load GloVe embeddings
embeddings_index = {}
with open('/kaggle/input/glove6/glove.6B.300d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Create embedding matrix
num_words = min(max_features, len(tokenizer.word_index) + 1)
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i >= max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Define the model using functional API
inputs = Input(shape=(maxlen,))
embedding_layer = Embedding(num_words, embedding_dim,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=maxlen,
                            trainable=False)(inputs)

# LSTM branch
lstm_branch = LSTM(lstm_units, dropout=0.2, recurrent_dropout=0.2)(embedding_layer)

# CNN branch
cnn_branch = Conv1D(filters, kernel_size, activation='relu')(embedding_layer)
cnn_branch = GlobalMaxPooling1D()(cnn_branch)

# Concatenate both branches
merged = Concatenate()([lstm_branch, cnn_branch])
output = Dense(1, activation='sigmoid')(merged)

# Create model
model = Model(inputs=inputs, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test, y_test)
print('Test accuracy:', accuracy)
end_time = time.time()

print("Time required to fine-tune: ", end_time - start_time)

20 epochs

In [ ]:
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, Conv1D, GlobalMaxPooling1D, Concatenate
from tensorflow.keras.initializers import Constant
import tensorflow as tf

# Record start time
start_time = time.time()

# Parameters for the model and training
max_features = 20000  # Number of words to consider as features
maxlen = 200  # Max length of individual reviews
embedding_dim = 300  # Dimension of word embeddings
lstm_units = 128  # Number of LSTM units
filters = 64  # Number of filters for CNN
kernel_size = 5  # Kernel size for CNN
batch_size = 32
epochs = 20  # Increase the number of epochs
early_stopping_patience = 3  # Early stopping patience

# Load train and test datasets from CSV files
train_data = pd.read_csv('/kaggle/input/plmdataset/train.csv/train.csv')
test_data = pd.read_csv('/kaggle/input/plmdataset/test.csv/test.csv')

# Preprocess train data
X_train = train_data['review'].values
y_train = train_data['sentiment'].values

# Preprocess test data
X_test = test_data['review'].values
y_test = test_data['sentiment'].values

# Convert labels to numerical format
y_train[y_train == 'negative'] = 0
y_train[y_train == 'positive'] = 1
y_train = y_train.astype(int)

y_test[y_test == 'negative'] = 0
y_test[y_test == 'positive'] = 1
y_test = y_test.astype(int)

# Tokenize and pad sequences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train)

X_train_tokenized = tokenizer.texts_to_sequences(X_train)
X_test_tokenized = tokenizer.texts_to_sequences(X_test)
X_train = pad_sequences(X_train_tokenized, maxlen=maxlen)
X_test = pad_sequences(X_test_tokenized, maxlen=maxlen)

# Load GloVe embeddings
embeddings_index = {}
with open('/kaggle/input/glove6/glove.6B.300d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Create embedding matrix
num_words = min(max_features, len(tokenizer.word_index) + 1)
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i >= max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Define the model using functional API
inputs = Input(shape=(maxlen,))
embedding_layer = Embedding(num_words, embedding_dim,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=maxlen,
                            trainable=False)(inputs)

# LSTM branch
lstm_branch = LSTM(lstm_units, dropout=0.2, recurrent_dropout=0.2)(embedding_layer)

# CNN branch
cnn_branch = Conv1D(filters, kernel_size, activation='relu')(embedding_layer)
cnn_branch = GlobalMaxPooling1D()(cnn_branch)

# Concatenate both branches
merged = Concatenate()([lstm_branch, cnn_branch])
output = Dense(1, activation='sigmoid')(merged)

# Create model
model = Model(inputs=inputs, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=early_stopping_patience, restore_best_weights=True
)

# Train the model with modified epochs and callbacks
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, 
          validation_split=0.2, callbacks=[early_stopping])

# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test, y_test)
print('Test accuracy:', accuracy)
end_time = time.time()

print("Time required to fine-tune: ", end_time - start_time)


l2 regularizatiom

In [ ]:
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, Conv1D, GlobalMaxPooling1D, Concatenate, Dropout
from tensorflow.keras.initializers import Constant
from tensorflow.keras import regularizers

# Record start time
start_time = time.time()

# Parameters for the model and training
max_features = 20000  # Number of words to consider as features
maxlen = 200  # Max length of individual reviews
embedding_dim = 300  # Dimension of word embeddings
lstm_units = 128  # Number of LSTM units
filters = 64  # Number of filters for CNN
kernel_size = 5  # Kernel size for CNN
batch_size = 32
epochs = 5  # Increase the number of epochs
early_stopping_patience = 3  # Early stopping patience
dropout_rate = 0.2  # Dropout rate for regularization

# Load train and test datasets from CSV files
train_data = pd.read_csv('/kaggle/input/plmdataset/train.csv/train.csv')
test_data = pd.read_csv('/kaggle/input/plmdataset/test.csv/test.csv')

# Preprocess train data
X_train = train_data['review'].values
y_train = train_data['sentiment'].values

# Preprocess test data
X_test = test_data['review'].values
y_test = test_data['sentiment'].values

# Convert labels to numerical format
y_train[y_train == 'negative'] = 0
y_train[y_train == 'positive'] = 1
y_train = y_train.astype(int)

y_test[y_test == 'negative'] = 0
y_test[y_test == 'positive'] = 1
y_test = y_test.astype(int)

# Tokenize and pad sequences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_train)

X_train_tokenized = tokenizer.texts_to_sequences(X_train)
X_test_tokenized = tokenizer.texts_to_sequences(X_test)
X_train = pad_sequences(X_train_tokenized, maxlen=maxlen)
X_test = pad_sequences(X_test_tokenized, maxlen=maxlen)

# Load GloVe embeddings
embeddings_index = {}
with open('/kaggle/input/glove6/glove.6B.300d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Create embedding matrix
num_words = min(max_features, len(tokenizer.word_index) + 1)
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i >= max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Define the model using functional API
inputs = Input(shape=(maxlen,))
embedding_layer = Embedding(num_words, embedding_dim,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=maxlen,
                            trainable=False)(inputs)

# LSTM branch with dropout
lstm_branch = LSTM(lstm_units, dropout=dropout_rate, recurrent_dropout=dropout_rate,
                   kernel_regularizer=regularizers.l2(0.001))(embedding_layer)

# CNN branch with dropout
cnn_branch = Conv1D(filters, kernel_size, activation='relu')(embedding_layer)
cnn_branch = GlobalMaxPooling1D()(cnn_branch)
cnn_branch = Dropout(rate=dropout_rate)(cnn_branch)

# Concatenate both branches
merged = Concatenate()([lstm_branch, cnn_branch])
output = Dense(1, activation='sigmoid')(merged)

# Create model
model = Model(inputs=inputs, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=early_stopping_patience, restore_best_weights=True
)

# Train the model with modified epochs and callbacks
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, 
          validation_split=0.2, callbacks=[early_stopping])

# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test, y_test)
print('Test accuracy:', accuracy)
end_time = time.time()

print("Time required to fine-tune: ", end_time - start_time)
